In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import matplotlib.pyplot as plt

In [2]:
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 30   # EarlyStopping pehle hi rok dega

TRAIN_DIR = r"C:\Users\shikh\AIML\plant\plant_disease_prediction\datasets\POTATO\train"
TEST_DIR = r"C:\Users\shikh\AIML\plant\plant_disease_prediction\datasets\POTATO\test"


In [3]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)


In [4]:
train_data = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

test_data = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)


Found 1721 images belonging to 3 classes.
Found 431 images belonging to 3 classes.


In [5]:
class_names = list(train_data.class_indices.keys())
class_names

['Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy']

In [6]:
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),   # Reduced dropout
    layers.Dense(3, activation='softmax')
])


In [7]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 26, 26, 128)      0

In [10]:
import os
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# 🔹 folder jahan model save hoga
SAVE_DIR = r"C:\Users\shikh\AIML\plant\plant_disease_prediction\trained_models"
os.makedirs(SAVE_DIR, exist_ok=True)

# 🔹 full model path

MODEL_PATH = os.path.join(SAVE_DIR, "potato_best_model.h5")

callbacks = [
    EarlyStopping(
        monitor='val_loss',
        patience=4,
        restore_best_weights=True
    ),

    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.3,
        patience=2
    ),

    ModelCheckpoint(
        filepath=MODEL_PATH,
        monitor='val_loss',
        save_best_only=True
    )
]



In [11]:
history = model.fit(
    train_data,
    epochs=EPOCHS,
    validation_data=test_data,
    callbacks=callbacks
)


Epoch 1/30
54/54 [==============================] - 61s 1s/step - loss: 0.8668 - accuracy: 0.5613 - val_loss: 0.7045 - val_accuracy: 0.7657 - lr: 1.0000e-04
Epoch 2/30
54/54 [==============================] - 56s 1s/step - loss: 0.6027 - accuracy: 0.7682 - val_loss: 0.4364 - val_accuracy: 0.8538 - lr: 1.0000e-04
Epoch 3/30
54/54 [==============================] - 58s 1s/step - loss: 0.4269 - accuracy: 0.8425 - val_loss: 0.4057 - val_accuracy: 0.8422 - lr: 1.0000e-04
Epoch 4/30
54/54 [==============================] - 56s 1s/step - loss: 0.3279 - accuracy: 0.8780 - val_loss: 0.2869 - val_accuracy: 0.9002 - lr: 1.0000e-04
Epoch 5/30
54/54 [==============================] - 58s 1s/step - loss: 0.2598 - accuracy: 0.8948 - val_loss: 0.3808 - val_accuracy: 0.8074 - lr: 1.0000e-04
Epoch 6/30
54/54 [==============================] - 68s 1s/step - loss: 0.2217 - accuracy: 0.9181 - val_loss: 0.5252 - val_accuracy: 0.7262 - lr: 1.0000e-04
Epoch 7/30
54/54 [==============================] - 59s 1s